In [1]:
import numpy as np
import pandas as pd
import string
import re
from IPython.core.display import HTML

# Gráficos
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

# Preprocesado y modelado
from collections import defaultdict
from sklearn import metrics
from nltk.corpus import stopwords
from sklearn.cluster import KMeans


# cambiador de formatos para factorizar un dataset
from sklearn.feature_extraction.text import TfidfVectorizer 

# control de rendimiento
from time import time

# Configuración warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
ruta = 'Opiniones.csv'
data = pd.read_csv(ruta)
# print(data.head())

In [3]:
display(HTML(data.sample(20).to_html()))

,AÑO,SEDE,FACULTAD,CARRERA,OBSERVACIONES
19361,2019.0,SEDE 1,FACULTAD 4,LICENCIATURA 14,Hace falta donde imprimir en el turno de la noche
42932,2019.0,SEDE 1,FACULTAD 5,LICENCIATURA 23,"Opción de acceso a herramientas Informática para compañeros que no se lo puedan permitir o por motivos ajenos, no cuenten con PC en un momento especí----o"
1516,2019.0,SEDE 4,FACULTAD 5,LICENCIATURA 21,.
9867,2019.0,SEDE 6,FACULTAD 1,INGENIERIA 01,"Cambiar algunas sillas de los salones que no están en buen estado (flojas, rotas en algunas partes, le faltan tornillos, etc)."
20634,2019.0,SEDE 1,FACULTAD 2,INGENIERIA 08,información de la convalidación
46344,2019.0,SEDE 1,FACULTAD 1,INGENIERIA 06,Revisar el estado de los equipos de laboratorio
51884,2019.0,SEDE 1,FACULTAD 3,LICENCIATURA 08,WiFi
6100,2019.0,SEDE 1,FACULTAD 1,LICENCIATURA 04,Agua fría en las fuentes
36014,2019.0,SEDE 6,FACULTAD 2,INGENIERIA 08,mejorar seguridad
42279,2019.0,SEDE 1,FACULTAD 1,INGENIERIA 01,no tengo observaciones


In [4]:
data.rename(columns={'AÑO':'YEAR'}, inplace=True)
print(data.head())

     YEAR    SEDE    FACULTAD          CARRERA OBSERVACIONES
0  2019.0  SEDE 2  FACULTAD 1  LICENCIATURA 03             0
1  2019.0  SEDE 2  FACULTAD 1  LICENCIATURA 04             0
2  2019.0  SEDE 2  FACULTAD 2  LICENCIATURA 07             0
3  2019.0  SEDE 2  FACULTAD 3    INGENIERIA 15             0
4  2019.0  SEDE 3  FACULTAD 1  LICENCIATURA 04             0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52583 entries, 0 to 52582
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   YEAR           52519 non-null  float64
 1   SEDE           52519 non-null  object 
 2   FACULTAD       52519 non-null  object 
 3   CARRERA        52338 non-null  object 
 4   OBSERVACIONES  52340 non-null  object 
dtypes: float64(1), object(4)
memory usage: 2.0+ MB


In [6]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52160 entries, 0 to 52515
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   YEAR           52160 non-null  float64
 1   SEDE           52160 non-null  object 
 2   FACULTAD       52160 non-null  object 
 3   CARRERA        52160 non-null  object 
 4   OBSERVACIONES  52160 non-null  object 
dtypes: float64(1), object(4)
memory usage: 2.4+ MB


In [7]:
data['YEAR'] = data['YEAR'].astype(float).astype(int)
col_list = ['YEAR','SEDE', 'FACULTAD', 'CARRERA']
for col in col_list:
    if col == 'TEXTO_TOKEN':
        data[col] = data[col].apply(lambda x: ' '.join(x))
    print(f'Columna {col}: {data[col].nunique()} subniveles')

Columna YEAR: 1 subniveles
Columna SEDE: 8 subniveles
Columna FACULTAD: 6 subniveles
Columna CARRERA: 103 subniveles


In [8]:
data.drop(columns='YEAR', inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52160 entries, 0 to 52515
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   SEDE           52160 non-null  object
 1   FACULTAD       52160 non-null  object
 2   CARRERA        52160 non-null  object
 3   OBSERVACIONES  52160 non-null  object
dtypes: object(4)
memory usage: 2.0+ MB


In [9]:
display(HTML(data.sample(10).to_html()))

,SEDE,FACULTAD,CARRERA,OBSERVACIONES
16694,SEDE 6,FACULTAD 5,LICENCIATURA 22,Espacio en biblioteca
44982,SEDE 1,FACULTAD 3,LICENCIATURA 10,Proyectores
2669,SEDE 1,FACULTAD 5,LICENCIATURA 23,.
15262,SEDE 1,FACULTAD 5,LICENCIATURA 23,El wifi no funciona en algunos pisos
24259,SEDE 1,FACULTAD 4,INGENIERIA 17,"Las filas de las cafeterias se hacen muy largas, podria mejorar"
51609,SEDE 1,FACULTAD 3,LICENCIATURA 08,Veri----ar los alimentos que venden con moho y vencidos
3902,SEDE 1,FACULTAD 4,INGENIERIA 21,.
5051,SEDE 1,FACULTAD 5,LICENCIATURA 22,A
13135,SEDE 4,FACULTAD 1,LICENCIATURA 03,El Centro de Fotocopiado. NO hay salones de estudio.
3659,SEDE 1,FACULTAD 3,LICENCIATURA 12,.


Acomodar para que quite stopwords

In [10]:
spanish_stopwords = list(stopwords.words('spanish'))
spanish_stopwords.extend(("xq", "oye", "dale", "dele", "ba", "abc", "nan", "na"))
print(spanish_stopwords)

['de', 'la', 'que', 'el', 'en', 'y', 'a', 'los', 'del', 'se', 'las', 'por', 'un', 'para', 'con', 'no', 'una', 'su', 'al', 'lo', 'como', 'más', 'pero', 'sus', 'le', 'ya', 'o', 'este', 'sí', 'porque', 'esta', 'entre', 'cuando', 'muy', 'sin', 'sobre', 'también', 'me', 'hasta', 'hay', 'donde', 'quien', 'desde', 'todo', 'nos', 'durante', 'todos', 'uno', 'les', 'ni', 'contra', 'otros', 'ese', 'eso', 'ante', 'ellos', 'e', 'esto', 'mí', 'antes', 'algunos', 'qué', 'unos', 'yo', 'otro', 'otras', 'otra', 'él', 'tanto', 'esa', 'estos', 'mucho', 'quienes', 'nada', 'muchos', 'cual', 'poco', 'ella', 'estar', 'estas', 'algunas', 'algo', 'nosotros', 'mi', 'mis', 'tú', 'te', 'ti', 'tu', 'tus', 'ellas', 'nosotras', 'vosotros', 'vosotras', 'os', 'mío', 'mía', 'míos', 'mías', 'tuyo', 'tuya', 'tuyos', 'tuyas', 'suyo', 'suya', 'suyos', 'suyas', 'nuestro', 'nuestra', 'nuestros', 'nuestras', 'vuestro', 'vuestra', 'vuestros', 'vuestras', 'esos', 'esas', 'estoy', 'estás', 'está', 'estamos', 'estáis', 'están', 'e

In [11]:
def limpiar_tokenizar(texto):
    # Eliminación de números
    nuevo_texto = re.sub("\d+", ' ', texto)
    # Se convierte todo el texto a minúsculas
    nuevo_texto = nuevo_texto.lower()
    # Eliminación de páginas web (palabras que empiezan por "http")
    nuevo_texto = re.sub('http\S+', ' ', nuevo_texto)
    # Eliminación de signos de puntuación
    regex = '[\\!\\"”\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\
    \\\.\\/\\:\\;\\<\\=\\>\\¿?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~—]'
    nuevo_texto = re.sub(regex , ' ', nuevo_texto)
    # cambio de - por nada para casos de palabras 
    nuevo_texto = re.sub('-', '', nuevo_texto)
    # Eliminación de espacios en blanco múltiples
    nuevo_texto = re.sub("\\s+", ' ', nuevo_texto)
    # Tokenización por palabras individuales
    nuevo_texto = nuevo_texto.split(sep = ' ')
    # Eliminación de tokens con una longitud < 2
    nuevo_texto = [token for token in nuevo_texto if len(token) > 2]
    # Eliminación de stopwords
    nuevo_texto = [token for token in nuevo_texto if token not in spanish_stopwords]
    # quitar los token que sean solamente de una letra repetidas 2 o mas veces
    nuevo_texto = [token for token in nuevo_texto if not re.fullmatch(r'(.)\1*', token)]
    
    return(nuevo_texto)

test = "Esto es 1 ejemplo de l'limpieza de6 TEXTO  https://t.co/rnHPgyhx4Z @cienciadedatos wi-fi #textmining zzzzz oooooo aaaaaaaa hhh eeeeeeee todo bien, bien"
print(test)
print(limpiar_tokenizar(texto=test))

Esto es 1 ejemplo de l'limpieza de6 TEXTO  https://t.co/rnHPgyhx4Z @cienciadedatos wi-fi #textmining zzzzz oooooo aaaaaaaa hhh eeeeeeee todo bien, bien
['ejemplo', 'limpieza', 'texto', 'cienciadedatos', 'wifi', 'textmining', 'bien', 'bien']


In [12]:
def join_tokens(tokens):
    if isinstance(tokens, list):
        return ' '.join(tokens)
    else:
        return ''

In [13]:
display(HTML(data.sample(10).to_html()))

,SEDE,FACULTAD,CARRERA,OBSERVACIONES
5912,SEDE 1,FACULTAD 3,LICENCIATURA 08,Adquirir otro método de pago en las cafeterías
26190,SEDE 6,FACULTAD 4,LICENCIATURA 14,los talleres se pueden modigicar mas
48821,SEDE 1,FACULTAD 5,LICENCIATURA 22,Soy del horario nocturno y es casi imposible hallar una caja abierta a las 6:15 pm para pagar la matricula
36866,SEDE 4,FACULTAD 5,LICENCIATURA 23,mejores lugares para estudiar
604,SEDE 1,FACULTAD 6,LICENCIATURA 24,-
33912,SEDE 7,FACULTAD 1,INGENIERIA 01,Mejorar la calidad de la comida
13421,SEDE 1,FACULTAD 5,INGENIERIA 25,El internet
48399,SEDE 1,FACULTAD 5,INGENIERIA 25,Sin comentarios
5981,SEDE 6,FACULTAD 2,INGENIERIA 08,Agradecería una mejor red WiFi.
21189,SEDE 1,FACULTAD 2,INGENIERIA 08,Internet


In [14]:
data['OBSERVACIONES'] = data['OBSERVACIONES'].astype(str)
data['TEXTO_TOKEN'] = data['OBSERVACIONES'].apply(lambda x: limpiar_tokenizar(x))

# Concatenar la lista de tokens en una sola cadena con espacios
# data['TEXTO_STRING'] = data['TEXTO_TOKEN'].apply(join_tokens)

# vectorizer = TfidfVectorizer(
#     max_df=0.5,
#     min_df=5,
# )
# t0 = time()
# X_tfidf = vectorizer.fit_transform(data['TEXTO_STRING'])

# print(f"vectorization done in {time() - t0:.3f} s")
# print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")

In [15]:
data = data[data['TEXTO_TOKEN'].apply(len) > 3]
print(f'Tamaño del set después de eliminar las filas con menos de 3 tokens: {data.shape}')
display(HTML(data.head(10).to_html()))

Tamaño del set después de eliminar las filas con menos de 3 tokens: (18113, 5)


,SEDE,FACULTAD,CARRERA,OBSERVACIONES,TEXTO_TOKEN
878,SEDE 4,FACULTAD 3,LICENCIATURA 10,"- Capacidad, iluminación, ventilación de las aulas. El aire se suele dañar Los salones se ven muy oscuros, en mi opinión no motivan a dar clase ahí. - Servicios de Fotocopiado. Se necesita urgente.","[capacidad, iluminación, ventilación, aulas, aire, suele, dañar, salones, ven, oscuros, opinión, motivan, dar, clase, ahí, servicios, fotocopiado, necesita, urgente]"
880,SEDE 1,FACULTAD 5,INGENIERIA 25,deberían modernizar los equipos utilizados en los laboratorios.,"[deberían, modernizar, equipos, utilizados, laboratorios]"
881,SEDE 1,FACULTAD 4,INGENIERIA 17,"---- en necesidad del remodelación interior, abrir mas entradas de estacionamientos (hay una que se construyo pero esta cerrada), mas puentes peatonales y reparar los artefactos de los baños.","[necesidad, remodelación, interior, abrir, mas, entradas, estacionamientos, construyo, cerrada, mas, puentes, peatonales, reparar, artefactos, baños]"
882,SEDE 1,FACULTAD 1,INGENIERIA 01,---- necesita cortinas para uso de proyectores,"[necesita, cortinas, uso, proyectores]"
883,SEDE 8,FACULTAD 1,INGENIERIA 01,no contamos con servicio de fotocopia. El Wi-Fi no llega a ciertas aulas en donde muchas veces se necesita para trabajos que nos asignan.,"[contamos, servicio, fotocopia, wifi, llega, ciertas, aulas, muchas, veces, necesita, trabajos, asignan]"
885,SEDE 8,FACULTAD 3,INGENIERIA 15,"WiFi no es suficiente, implementar un WiFi con mayor capacidad de banda ancha y mayores velocidades sería oportuno.","[wifi, suficiente, implementar, wifi, mayor, capacidad, banda, ancha, mayores, velocidades, oportuno]"
893,SEDE 1,FACULTAD 2,TÉCNICO 05,#14 / pocas charlas o seminarios de innovación de información enfocado para el grupo nocturno,"[pocas, charlas, seminarios, innovación, información, enfocado, grupo, nocturno]"
894,SEDE 1,FACULTAD 2,POSTGRADO 20,#14 Informar sobre Servicios de Orientación para el horario estudiantil nocturno.,"[informar, servicios, orientación, horario, estudiantil, nocturno]"
895,SEDE 1,FACULTAD 2,POSTGRADO 20,#18 Mejorar la orientación y informar a tiempo sobre materias y diferentes procedimientos de matrícula.,"[mejorar, orientación, informar, tiempo, materias, diferentes, procedimientos, matrícula]"
896,SEDE 1,FACULTAD 2,POSTGRADO 20,"#19 La Dirección de Finanzas debe ser absorbida por la Secretaria general y analizar herramientas para agilizar pagos, reclamos, etc.","[dirección, finanzas, debe, ser, absorbida, secretaria, general, analizar, herramientas, agilizar, pagos, reclamos, etc]"


In [16]:
import spacy
import pandas as pd

# Cargar el modelo en español
nlp = spacy.load("es_core_news_sm")

# Función para lematizar una palabra o frase
def lemmatize_text(text):
    
    if isinstance(text, list):
        text = ' '.join(text)
    doc = nlp(text)
    return [token.lemma_ for token in doc if not token.is_stop]


# Tomar una muestra aleatoria de 10 palabras del DataFrame
# prueba = data.reset_index(drop=True)
# display(HTML(prueba.to_html()))

# Aplicar la función de lematización a la columna 'TEXTO_TOKEN' y agregar una nueva columna con los resultados
data['LEMMA'] = data['TEXTO_TOKEN'].apply(lemmatize_text)

# print(prueba)
display(HTML(data.sample(10).to_html()))


,SEDE,FACULTAD,CARRERA,OBSERVACIONES,TEXTO_TOKEN,LEMMA
5581,SEDE 1,FACULTAD 1,LICENCIATURA 02,Aconsejar mejor sobre la convalidación de materias para realizarlo a tiempo.,"[aconsejar, mejor, convalidación, materias, realizarlo, tiempo]","[aconsejar, convalidación, materia, realizar él, tiempo]"
29270,SEDE 1,FACULTAD 1,INGENIERIA 06,Mayor agilidad con las cajas en cafeterías,"[mayor, agilidad, cajas, cafeterías]","[agilidad, caja, cafetería]"
25892,SEDE 6,FACULTAD 2,INGENIERIA 08,Los precios en el cafetín son demasiado elevados,"[precios, cafetín, demasiado, elevados]","[precio, cafetín, elevado]"
24079,SEDE 1,FACULTAD 5,INGENIERIA 29,Las cafeterías siempre están llenas y se hacen filas muy largas.,"[cafeterías, siempre, llenas, hacen, filas, largas]","[cafetería, llena, fila, larga]"
33447,SEDE 1,FACULTAD 5,POSTGRADO 36,Mejorar en los pagos vía web,"[mejorar, pagos, vía, web]","[mejorar, pago, vía, web]"
47125,SEDE 6,FACULTAD 4,LICENCIATURA 15,Se necesita mayor cantidad de estacionamientos.,"[necesita, mayor, cantidad, estacionamientos]","[necesitar, cantidad, estacionamiento]"
47570,SEDE 1,FACULTAD 5,INGENIERIA 23,Seguridad en las paradas de buses de la universidad,"[seguridad, paradas, buses, universidad]","[seguridad, parada, bús, universidad]"
15773,SEDE 1,FACULTAD 5,INGENIERIA 23,"En el salón debería haber mantenimiento cuando algún material se daña por alguna razón, ya que en un salón de laboratorio se nos dañó algunos materiales y nunca lo arreglaron.","[salón, debería, haber, mantenimiento, algún, material, daña, alguna, razón, salón, laboratorio, dañó, materiales, nunca, arreglaron]","[salón, deber, mantenimiento, material, dañar, razón, salón, laboratorio, dañar, material, arreglar]"
16460,SEDE 1,FACULTAD 2,TÉCNICO 03,Equipos de laboratorio electrónico muestran algunas fallas,"[equipos, laboratorio, electrónico, muestran, fallas]","[equipo, laboratorio, electrónico, mostrar, falla]"
14454,SEDE 1,FACULTAD 4,INGENIERIA 21,"el punto de encuentro al lado de la biblioteca aveces esta cerrado, entiendo que se haga mantenimiento, pero en ciertas ocasiones no veo la necesidad de evitar el paso o usar asientos de esta zona","[punto, encuentro, lado, biblioteca, aveces, cerrado, entiendo, haga, mantenimiento, ciertas, ocasiones, veo, necesidad, evitar, paso, usar, asientos, zona]","[punto, encuentro, biblioteca, avez, cerrado, entiendo, hacer, mantenimiento, ocasión, ver, necesidad, evitar, paso, asiento, zón]"


Mostrar a los pelaos

In [17]:
import re
from collections import Counter

# Lee el texto y procesa las palabras
def words(text):
    return re.findall(r'\w+', text.lower())

# Carga y cuenta las palabras en el archivo
WORDS = Counter(words(open('big.txt', encoding='utf-8').read()))

# Función de probabilidad de la palabra
def P(word, N=sum(WORDS.values())): 
    return WORDS[word] / N

# Función de corrección
def correction(word): 
    return max(candidates(word), key=P)

# Genera posibles correcciones
def candidates(word): 
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

# Subconjunto de palabras conocidas
def known(words): 
    return set(w for w in words if w in WORDS)

# Ediciones a una distancia
def edits1(word):
    letters    = 'abcdefghijklmnopqrstuvwxyzáéíóúñ'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

# Ediciones a dos distancias
def edits2(word): 
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

# Ejemplo de uso
print(correction("árbol"))  # Debería sugerir "árbol" si es correcto


árbol


In [18]:
# reinicia los index de 0 en adelante
prueba = data.reset_index(drop=True)[:5000]

def process_words(word_list):
    if not word_list:
        return []
    first_word = word_list[0]
    if first_word not in WORDS:
        corrected_word = correction(first_word)
        return [corrected_word] + process_words(word_list[1:])
    else:
        return [first_word] + process_words(word_list[1:])
    
prueba['CORRECION'] = prueba['LEMMA'].apply(process_words)
display(HTML(prueba.head(10).to_html()))

,SEDE,FACULTAD,CARRERA,OBSERVACIONES,TEXTO_TOKEN,LEMMA,CORRECION
0,SEDE 4,FACULTAD 3,LICENCIATURA 10,"- Capacidad, iluminación, ventilación de las aulas. El aire se suele dañar Los salones se ven muy oscuros, en mi opinión no motivan a dar clase ahí. - Servicios de Fotocopiado. Se necesita urgente.","[capacidad, iluminación, ventilación, aulas, aire, suele, dañar, salones, ven, oscuros, opinión, motivan, dar, clase, ahí, servicios, fotocopiado, necesita, urgente]","[capacidad, iluminación, ventilación, aula, aire, soler, dañar, salón, ver, oscuro, opinión, motivar, clase, servicio, fotocopiado, necesitar, urgente]","[capacidad, iluminación, ventilación, aula, aire, soler, dañar, salón, ver, oscuro, opinión, motivar, clase, servicio, fotocopiado, necesitar, urgente]"
1,SEDE 1,FACULTAD 5,INGENIERIA 25,deberían modernizar los equipos utilizados en los laboratorios.,"[deberían, modernizar, equipos, utilizados, laboratorios]","[deber, modernizar, equipo, utilizado, laboratorio]","[deber, modernizar, equipo, utilizado, laboratorio]"
2,SEDE 1,FACULTAD 4,INGENIERIA 17,"---- en necesidad del remodelación interior, abrir mas entradas de estacionamientos (hay una que se construyo pero esta cerrada), mas puentes peatonales y reparar los artefactos de los baños.","[necesidad, remodelación, interior, abrir, mas, entradas, estacionamientos, construyo, cerrada, mas, puentes, peatonales, reparar, artefactos, baños]","[necesidad, remodelación, interior, abrir, entrada, estacionamiento, construyo, cerrado, puente, peatonal, reparar, artefacto, baño]","[necesidad, remodelación, interior, abrir, entrada, estacionamiento, construyo, cerrado, puente, peatonal, reparar, artefacto, baño]"
3,SEDE 1,FACULTAD 1,INGENIERIA 01,---- necesita cortinas para uso de proyectores,"[necesita, cortinas, uso, proyectores]","[necesitar, cortina, proyector]","[necesitar, cortina, proyector]"
4,SEDE 8,FACULTAD 1,INGENIERIA 01,no contamos con servicio de fotocopia. El Wi-Fi no llega a ciertas aulas en donde muchas veces se necesita para trabajos que nos asignan.,"[contamos, servicio, fotocopia, wifi, llega, ciertas, aulas, muchas, veces, necesita, trabajos, asignan]","[contar, servicio, fotocopio, wifi, llegar, aula, necesitar, trabajo, asignar]","[contar, servicio, fotocopio, wifi, llegar, aula, necesitar, trabajo, asignar]"
5,SEDE 8,FACULTAD 3,INGENIERIA 15,"WiFi no es suficiente, implementar un WiFi con mayor capacidad de banda ancha y mayores velocidades sería oportuno.","[wifi, suficiente, implementar, wifi, mayor, capacidad, banda, ancha, mayores, velocidades, oportuno]","[wifi, suficiente, implementar, wifi, capacidad, bando, ancho, mayor, velocidad, oportuno]","[wifi, suficiente, implementar, wifi, capacidad, bando, ancho, mayor, velocidad, oportuno]"
6,SEDE 1,FACULTAD 2,TÉCNICO 05,#14 / pocas charlas o seminarios de innovación de información enfocado para el grupo nocturno,"[pocas, charlas, seminarios, innovación, información, enfocado, grupo, nocturno]","[charla, seminario, innovación, información, enfocado, grupo, nocturno]","[charla, seminario, innovación, información, enfocado, grupo, nocturno]"
7,SEDE 1,FACULTAD 2,POSTGRADO 20,#14 Informar sobre Servicios de Orientación para el horario estudiantil nocturno.,"[informar, servicios, orientación, horario, estudiantil, nocturno]","[informar, servicio, orientación, horario, estudiantil, nocturno]","[informar, servicio, orientación, horario, estudiantil, nocturno]"
8,SEDE 1,FACULTAD 2,POSTGRADO 20,#18 Mejorar la orientación y informar a tiempo sobre materias y diferentes procedimientos de matrícula.,"[mejorar, orientación, informar, tiempo, materias, diferentes, procedimientos, matrícula]","[mejorar, orientación, informar, tiempo, materia, procedimiento, matrículo]","[mejorar, orientación, informar, tiempo, materia, procedimiento, matriculo]"
9,SEDE 1,FACULTAD 2,POSTGRADO 20,"#19 La Dirección de Finanzas debe ser absorbida por la Secretaria general y analizar herramientas para agilizar pagos, reclamos, etc.","[di

In [19]:
import gensim
processed_docs = []

for doc in prueba['CORRECION']:
    processed_docs.append(doc)
print(processed_docs[:2])
dictionary = gensim.corpora.Dictionary(processed_docs)

[['capacidad', 'iluminación', 'ventilación', 'aula', 'aire', 'soler', 'dañar', 'salón', 'ver', 'oscuro', 'opinión', 'motivar', 'clase', 'servicio', 'fotocopiado', 'necesitar', 'urgente'], ['deber', 'modernizar', 'equipo', 'utilizado', 'laboratorio']]


In [20]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 aire
1 aula
2 capacidad
3 clase
4 dañar
5 fotocopiado
6 iluminación
7 motivar
8 necesitar
9 opinión
10 oscuro


In [21]:
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)
print(dictionary)

Dictionary<340 unique tokens: ['aire', 'aula', 'capacidad', 'clase', 'fotocopiado']...>


In [22]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print(bow_corpus[:2])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)], [(11, 1), (12, 1), (13, 1)]]


In [23]:
document_num = 20
bow_doc_x = bow_corpus[document_num]
print(bow_doc_x)
for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

[(30, 1), (76, 1), (77, 1), (78, 1), (79, 1)]
Word 30 ("información") appears 1 time.
Word 76 ("divulgación") appears 1 time.
Word 77 ("favor") appears 1 time.
Word 78 ("institucional") appears 1 time.
Word 79 ("libre") appears 1 time.


In [24]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 8, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [25]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Topic: 0 
Words: 0.066*"pago" + 0.064*"cafetee" + 0.050*"precio" + 0.042*"cafeteria" + 0.039*"área" + 0.030*"alimento" + 0.027*"comida" + 0.024*"estudiante" + 0.022*"sitio" + 0.021*"variedad"


Topic: 1 
Words: 0.082*"salón" + 0.054*"espacio" + 0.052*"estudio" + 0.048*"facultad" + 0.039*"estacionamiento" + 0.038*"biblioteca" + 0.027*"deber" + 0.026*"virtual" + 0.026*"colocar" + 0.022*"materia"


Topic: 2 
Words: 0.117*"estudiante" + 0.067*"cafetera" + 0.066*"menú" + 0.057*"deber" + 0.035*"wifi" + 0.032*"estudiantil" + 0.029*"cantidad" + 0.024*"mejorar" + 0.023*"red" + 0.022*"conectar"


Topic: 3 
Words: 0.132*"servicio" + 0.092*"mejorar" + 0.035*"deber" + 0.035*"centro" + 0.030*"noche" + 0.026*"fotocopiado" + 0.025*"lugar" + 0.023*"impresión" + 0.022*"caer" + 0.022*"universidad"


Topic: 4 
Words: 0.069*"sistema" + 0.051*"tiempo" + 0.049*"matriculo" + 0.040*"ocasión" + 0.037*"proceso" + 0.029*"hora" + 0.029*"materia" + 0.029*"copia" + 0.028*"sacar" + 0.023*"matricular"


Topic: 5 
Word

In [31]:
import random
num = prueba.index[random.randint(0, prueba.shape[0])]
unseen_document = prueba["LEMMA"][num]
print(unseen_document)

['deber', 'cambiar', 'actualizar', 'equipo', 'laboratorio']


In [32]:
bow_vector = dictionary.doc2bow(unseen_document)

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.6202278733253479	 Topic: 0.163*"laboratorio" + 0.072*"equipo" + 0.042*"señal" + 0.035*"edificio" + 0.034*"computadora"
Score: 0.2546443045139313	 Topic: 0.082*"salón" + 0.054*"espacio" + 0.052*"estudio" + 0.048*"facultad" + 0.039*"estacionamiento"
Score: 0.020874077454209328	 Topic: 0.132*"servicio" + 0.092*"mejorar" + 0.035*"deber" + 0.035*"centro" + 0.030*"noche"
Score: 0.020864224061369896	 Topic: 0.117*"estudiante" + 0.067*"cafetera" + 0.066*"menú" + 0.057*"deber" + 0.035*"wifi"
Score: 0.020856689661741257	 Topic: 0.082*"clase" + 0.056*"aire" + 0.050*"salón" + 0.043*"aula" + 0.042*"acondicionado"
Score: 0.020845379680395126	 Topic: 0.227*"wifi" + 0.095*"funcionar" + 0.080*"lento" + 0.060*"llegar" + 0.043*"universidad"
Score: 0.020844096317887306	 Topic: 0.066*"pago" + 0.064*"cafetee" + 0.050*"precio" + 0.042*"cafeteria" + 0.039*"área"
Score: 0.020843394100666046	 Topic: 0.069*"sistema" + 0.051*"tiempo" + 0.049*"matriculo" + 0.040*"ocasión" + 0.037*"proceso"


In [33]:
# Obtener la representación BoW del documento
bow_vector = dictionary.doc2bow(unseen_document)

# Obtener la asignación de temas para el documento
lda_scores = sorted(lda_model[bow_vector], key=lambda tup: -1 * tup[1])

# Obtener el tema con la puntuación más alta
primary_topic = lda_scores[0][0] if lda_scores else None

# Mostrar la puntuación y el tema asignado
for index, score in lda_scores:
    print(f"Score: {score}\t Topic: {index}: {lda_model.print_topic(index, 5)}")

if primary_topic is not None:
    print(f"\nEl tema más representativo es el tema {primary_topic} con la puntuación más alta.")
else:
    print("\nNo se asignó ningún tema al documento.")

Score: 0.6201954483985901	 Topic: 6: 0.163*"laboratorio" + 0.072*"equipo" + 0.042*"señal" + 0.035*"edificio" + 0.034*"computadora"
Score: 0.2546767294406891	 Topic: 1: 0.082*"salón" + 0.054*"espacio" + 0.052*"estudio" + 0.048*"facultad" + 0.039*"estacionamiento"
Score: 0.020874060690402985	 Topic: 3: 0.132*"servicio" + 0.092*"mejorar" + 0.035*"deber" + 0.035*"centro" + 0.030*"noche"
Score: 0.020864220336079597	 Topic: 2: 0.117*"estudiante" + 0.067*"cafetera" + 0.066*"menú" + 0.057*"deber" + 0.035*"wifi"
Score: 0.02085667848587036	 Topic: 7: 0.082*"clase" + 0.056*"aire" + 0.050*"salón" + 0.043*"aula" + 0.042*"acondicionado"
Score: 0.02084537222981453	 Topic: 5: 0.227*"wifi" + 0.095*"funcionar" + 0.080*"lento" + 0.060*"llegar" + 0.043*"universidad"
Score: 0.020844096317887306	 Topic: 0: 0.066*"pago" + 0.064*"cafetee" + 0.050*"precio" + 0.042*"cafeteria" + 0.039*"área"
Score: 0.020843394100666046	 Topic: 4: 0.069*"sistema" + 0.051*"tiempo" + 0.049*"matriculo" + 0.040*"ocasión" + 0.037*"pr

In [34]:
topic_to_top_word = {}

# Iterar sobre los temas en el modelo LDA
for idx, topic in lda_model.print_topics(-1):
    # Dividir el tema en palabras y pesos
    words_weights = topic.split(" + ")
    # Extraer las palabras y sus pesos
    words_weights = [(float(weight.split('*')[0]), weight.split('*')[1].strip('"')) for weight in words_weights]
    # Filtrar las palabras con un peso mayor a 0.1
    words_weights = [ww for ww in words_weights if ww[0] > 0.09]
    if words_weights:
        # Seleccionar la palabra con el mayor peso
        top_word = max(words_weights, key=lambda ww: ww[0])
        # Asignar la palabra con el mayor peso al tema en el diccionario
        topic_to_top_word[idx] = top_word[1]

# Mostrar el diccionario
print(topic_to_top_word)

{2: 'estudiante', 3: 'servicio', 5: 'wifi', 6: 'laboratorio'}


In [30]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    # print("\n")

prueba['SCORE'] = ""
prueba['TOPIC'] = ""
prueba['name_topic'] = ""
for i in range(len(prueba)):
    num = prueba.index[i]
    unseen_document = prueba["LEMMA"][num]
    bow_vector = dictionary.doc2bow(unseen_document)
    lda_scores = sorted(lda_model[bow_vector], key=lambda tup: -1 * tup[1])
    primary_topic = lda_scores[0][0] if lda_scores else None
    # print(f"Documento {num}")
    # print(f"Texto: {unseen_document}")
    # print(f"Puntuación y tema asignado: {lda_scores}")
    # print(f"Tema más representativo: {primary_topic}")
    # print("\n")
    prueba["TOPIC"][i] = primary_topic
    if primary_topic in topic_to_top_word:
        prueba["name_topic"][i] = topic_to_top_word[primary_topic]
    # como hago para que lda_scores[0][1] solo guarde 3 decimales?
    prueba["SCORE"][i] = f"{lda_scores[0][1]*100:.2f}%"

display(HTML(prueba[["SCORE","TOPIC","name_topic","LEMMA", "OBSERVACIONES"]].to_html()))


Topic: 0 
Words: 0.066*"pago" + 0.064*"cafetee" + 0.050*"precio" + 0.042*"cafeteria" + 0.039*"área" + 0.030*"alimento" + 0.027*"comida" + 0.024*"estudiante" + 0.022*"sitio" + 0.021*"variedad"
Topic: 1 
Words: 0.082*"salón" + 0.054*"espacio" + 0.052*"estudio" + 0.048*"facultad" + 0.039*"estacionamiento" + 0.038*"biblioteca" + 0.027*"deber" + 0.026*"virtual" + 0.026*"colocar" + 0.022*"materia"
Topic: 2 
Words: 0.117*"estudiante" + 0.067*"cafetera" + 0.066*"menú" + 0.057*"deber" + 0.035*"wifi" + 0.032*"estudiantil" + 0.029*"cantidad" + 0.024*"mejorar" + 0.023*"red" + 0.022*"conectar"
Topic: 3 
Words: 0.132*"servicio" + 0.092*"mejorar" + 0.035*"deber" + 0.035*"centro" + 0.030*"noche" + 0.026*"fotocopiado" + 0.025*"lugar" + 0.023*"impresión" + 0.022*"caer" + 0.022*"universidad"
Topic: 4 
Words: 0.069*"sistema" + 0.051*"tiempo" + 0.049*"matriculo" + 0.040*"ocasión" + 0.037*"proceso" + 0.029*"hora" + 0.029*"materia" + 0.029*"copia" + 0.028*"sacar" + 0.023*"matricular"
Topic: 5 
Words: 0.227*"

,SCORE,TOPIC,name_topic,LEMMA,OBSERVACIONES
0,63.48%,7,,"[capacidad, iluminación, ventilación, aula, aire, soler, dañar, salón, ver, oscuro, opinión, motivar, clase, servicio, fotocopiado, necesitar, urgente]","- Capacidad, iluminación, ventilación de las aulas. El aire se suele dañar Los salones se ven muy oscuros, en mi opinión no motivan a dar clase ahí. - Servicios de Fotocopiado. Se necesita urgente."
1,78.10%,6,laboratorio,"[deber, modernizar, equipo, utilizado, laboratorio]",deberían modernizar los equipos utilizados en los laboratorios.
2,58.92%,1,,"[necesidad, remodelación, interior, abrir, entrada, estacionamiento, construyo, cerrado, puente, peatonal, reparar, artefacto, baño]","---- en necesidad del remodelación interior, abrir mas entradas de estacionamientos (hay una que se construyo pero esta cerrada), mas puentes peatonales y reparar los artefactos de los baños."
3,39.88%,7,,"[necesitar, cortina, proyector]",---- necesita cortinas para uso de proyectores
4,49.62%,5,wifi,"[contar, servicio, fotocopio, wifi, llegar, aula, necesitar, trabajo, asignar]",no contamos con servicio de fotocopia. El Wi-Fi no llega a ciertas aulas en donde muchas veces se necesita para trabajos que nos asignan.
5,39.78%,5,wifi,"[wifi, suficiente, implementar, wifi, capacidad, bando, ancho, mayor, velocidad, oportuno]","WiFi no es suficiente, implementar un WiFi con mayor capacidad de banda ancha y mayores velocidades sería oportuno."
6,33.49%,7,,"[charla, seminario, innovación, información, enfocado, grupo, nocturno]",#14 / pocas charlas o seminarios de innovación de información enfocado para el grupo nocturno
7,51.72%,7,,"[informar, servicio, orientación, horario, estudiantil, nocturno]",#14 Informar sobre Servicios de Orientación para el horario estudiantil nocturno.
8,53.25%,4,,"[mejorar, orientación, informar, tiempo, materia, procedimiento, matrículo]",#18 Mejorar la orientación y informar a tiempo sobre materias y diferentes procedimientos de matrícula.
9,44.64%,3,servicio,"[dirección, finanza, absorbido, secretaria, general, analizar, herramienta, agilizar, pago, reclamo, etc]","#19 La Dirección de Finanzas debe ser absorbida por la Secretaria general y analizar herramientas para agilizar pagos, reclamos, etc."
